# AP 데이터 건물별 분할
Link: [APs by Buildings](https://pusanackr.sharepoint.com/:f:/s/INCLabPNUKorea/En6Ld5JZ-OVNpL7AwW37yM0B0BasoXzumtQfg2oKEwe8kg?e=icE1Vx, "test")
- 주의 사항
  - 공유중인 MS 원드라이브는 학교에서 제공하는 서비스입니다. 공유에 대한 제약사항으로 pusan.ac.kr 계정에 대해서만 접근이 가능합니다. 불편하더라도 학교 계정을 발급받아 접근해보기 바랍니다.
  
- 구성
   - 전체 데이터 : clients_ver1.1.csv
   - 건물별 분할 데이터 : AP_Dataset_by_building > 건물이름.csv

In [1]:
import csv
import json
import requests
import pprint

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# 데이터 경로는 환경에 따라 바뀔 수 있습니다.
result_df = pd.read_csv("./AP_Dataset_by_builidng/JeDo.csv", header = [0,1,2], index_col = 0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (17,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
result_df.head()

a/n/ac                                        \
           80211 Counters                                         
        ACK Failure Count FCS Error Count Frame Duplicate Count   
7148304             15.00          922.00                  0.00   
7148305              0.00          140.00                  0.00   
7148306              0.00           63.00                  0.00   
7148307              0.00           55.00                  0.00   
7148308              0.00          103.00                  0.00   

                                                           \
                                                            
        Multicast Rx Frame Count Multicast Tx Frame Count   
7148304                     0.00                   120.00   
7148305                     0.00                   117.00   
7148306                     0.00                   116.00   
7148307                     0.00                   117.00   
7148308                     0.00                   121.00   

                                                                              \
                                                                               
        Multiple Retry Count RTS Failure Count RTS Success Count Retry Count   
7148304                25.00              1.00             20.00       54.00   
7148305                 1.00              0.00              0.00        8.00   
7148306                19.00             40.00             54.00       89.00   
7148307                 0.00              0.00              0.00        0.00   
7148308                 0.00              0.00              0.00        0.00   

                                                                            \
                                                                             
        Rx Fragment Count Tx Failed Count Tx Fragment Count Tx Frame Count   
7148304              0.00           12.00             63.00         183.00   
7148305              0.00            3.00              6.00         123.00   
7148306              0.00           48.00            118.00         234.00   
7148307              0.00            1.00              1.00         117.00   
7148308              0.00            0.00              0.00         121.00   

                                                                   \
                                   Air Quality                      
        WEP Undecryptable Count Channel Number Interferer Count     
7148304                    0.00          48.00               0.00   
7148305                    0.00          48.00               0.00   
7148306                    1.00          48.00               0.00   
7148307                    0.00          48.00               0.00   
7148308                    0.00          48.00               0.00   

                                                                     \
                                                                      
            Most Recent Reported Time Total Interference Duty Cycle   
7148304  Fri May 31 23:26:36 GMT 2019                          0.00   
7148305  Sat Jun 01 00:28:10 GMT 2019                          0.00   
7148306  Sat Jun 01 01:28:40 GMT 2019                          0.00   
7148307  Sat Jun 01 02:29:08 GMT 2019                          0.00   
7148308  Sat Jun 01 03:29:36 GMT 2019                          0.00   

                                                                 \
                                                                  
        Total Interference Power (dBm)  Worst Air Quality Value   
7148304                          -71.00                   99.00   
7148305                          -71.00                   99.00   
7148306                          -72.00                   99.00   
7148307                          -66.00                   99.00   
7148308                          -73.00                   99.00   

                                                                    

In [5]:
set(result_df[('common','Info','AP Name')].values)

{'11S-201-1-3',
 '11S-201-1-4',
 '11S-201-2-5',
 '16K-201-1-1',
 '16K-201-1-2',
 '16K-201-2-1',
 '16K-201-2-2',
 '16K-201-2-3',
 '16K-201-2-4',
 '16K-201-3-1',
 '16K-201-3-2',
 '16K-201-3-3',
 '16K-201-3-4',
 '16K-201-4-1',
 '16K-201-4-2',
 '16K-201-4-3',
 '16K-201-4-4',
 '16K-201-4-5',
 '16K-201-5-1',
 '16K-201-5-2',
 '16K-201-5-3',
 'SAP-66.233',
 'SAP-66.235',
 'SAP-66.236',
 'SAP-66.240'}

# 장비 맥 관련 사항

- 같은 MAC Address와 Vendor이나 Endpoint Type이 다른 경우
> Data Cleansing : 같은 맥에 대해서 최빈값으로 치환하자
<img src='./images/mac1.PNG'/>
<img src='./images/mac2.PNG'/>

In [6]:
clients_df = pd.read_csv("clients.csv", header = [0,1], index_col = 0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
clients_df.head()

l1            AP                                     \
l2          Name        MAC Address      IP Address   
0   16AP-420-3-6  00:c8:8b:99:3c:40  172.20.140.136   
1   16AP-420-3-6  00:c8:8b:99:3c:40  172.20.140.136   
2   16AP-420-3-6  00:c8:8b:99:3c:40  172.20.140.136   
3   16AP-420-3-6  00:c8:8b:99:3c:40  172.20.140.136   
4   16AP-420-3-6  00:c8:8b:99:3c:40  172.20.140.136   

l1                                         Client                            \
l2                    Map Location     IP Address MAC Address    Radio Type   
0   Pusan > Lib-2 (420) > Lib-2-3F   172.21.26.30           0   802.11b/g/n   
1   Pusan > Lib-2 (420) > Lib-2-3F  172.21.24.103           1   802.11b/g/n   
2   Pusan > Lib-2 (420) > Lib-2-3F  172.21.26.200           2   802.11b/g/n   
3   Pusan > Lib-2 (420) > Lib-2-3F   10.29.101.41           2  802.11a/n/ac   
4   Pusan > Lib-2 (420) > Lib-2-3F  172.21.26.200           2   802.11b/g/n   

l1                                                                   \
l2      SSID VLAN ID         Protocol Policy Type              Type   
0   PNU-WiFi      17  802.11n(2.4GHz)        WPA2  Unified Wireless   
1   PNU-WiFi      17  802.11n(2.4GHz)        WPA2  Unified Wireless   
2    eduroam      17  802.11n(2.4GHz)        WPA2  Unified Wireless   
3    eduroam      17         802.11ac        WPA2  Unified Wireless   
4    eduroam      17  802.11n(2.4GHz)        WPA2  Unified Wireless   

l1                                                                     \
l2      Endpoint Type Association ID Authentication Encryption Cipher   
0   Linux-Workstation              1            Yes        CCMP (AES)   
1            iPad11,3              1            Yes        CCMP (AES)   
2   Linux-Workstation             17            Yes        CCMP (AES)   
3   Linux-Workstation             94            Yes        CCMP (AES)   
4   Linux-Workstation              4            Yes        CCMP (AES)   

l1                             Utilization                            \
l2 EAP Type Avg. Session Throughput (Kbps) Bytes Sent Bytes Received   
0      PEAP                            2.9      43908          67148   
1      TTLS                           <0.1          0              0   
2      PEAP                           46.1     877159        4333234   
3      PEAP                           <0.1          0              0   
4      PEAP                           <0.1          0              0   

l1                                                                             \
l2 Packets Sent Packets Received SNR (dB) RSSI (dBm) Data Retries RTS Retries   
0           294              215       26        -52          601           0   
1             0                0        2        -76           60           0   
2          3410             4128       13        -69         9352           0   
3             0                0       22        -69            3           0   
4             0                0        9        -64        10630           0   

l1 Association Time                              Disassociation Time        \
l2             year month day hour minute second                year month   
0              2020     4  18   11     32     15                2020     4   
1              2020     4  13   16      8     19                2020     4   
2              2020     2  10   14     46     48                2020     2   
3              2020     2  11   19     32     13                2020     2   
4              2020     2  11   19     47     17                2020     2   

l1                        Session Duration                        
l2 day hour minute second             days hours minutes seconds  
0   18   11     37     16                0     0       5       1  
1   13   16     13     20                0     0       5       1  
2   10   15      1     52                0     0      15       4  
3   11   19     42     15                0     0      10       2  
4   11   19     52     18   

In [8]:
len(list(set(clients_df[('Client', 'MAC Address')].values)))

48852

# 클라이언트 데이터 추가 확보

Link: [Clients by Buildings](https://pusanackr.sharepoint.com/:f:/s/INCLabPNUKorea/En6Ld5JZ-OVNpL7AwW37yM0B0BasoXzumtQfg2oKEwe8kg?e=icE1Vx, "test")
- 주의 사항
  - 공유중인 MS 원드라이브는 학교에서 제공하는 서비스입니다. 공유에 대한 제약사항으로 pusan.ac.kr 계정에 대해서만 접근이 가능합니다. 불편하더라도 학교 계정을 발급받아 접근해보기 바랍니다.
  
- 구성
   - 전체 데이터 : clients_ver1.1.csv
   - 건물별 분할 데이터 : Clients_Dataset_by_building > client_건물이름.csv

## 총 데이터 요약

In [3]:
clients_df = pd.read_csv("./Utility/clients_ver1.1.csv", header = [0,1], index_col = 0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
clients_df.describe()

l1    Association Time                                                         \
l2                 day       hour     minute      month     second       year   
count       9775562.00 9775562.00 9775562.00 9775562.00 9775562.00 9775562.00   
mean             16.78      13.73      29.92       7.17      29.40    2019.17   
std               8.98       4.40      17.60       2.93      17.34       0.37   
min               1.00       0.00       0.00       1.00       0.00    2019.00   
25%               9.00      11.00      15.00       5.00      14.00    2019.00   
50%              18.00      14.00      30.00       7.00      29.00    2019.00   
75%              25.00      17.00      46.00      10.00      44.00    2019.00   
max              31.00      23.00      59.00      12.00      59.00    2020.00   

l1            Client                        Disassociation Time             \
l2    Association ID MAC Address    VLAN ID                 day       hour   
count     9775562.00  9775562.00 9775562.00          9775562.00 9775562.00   
mean           21.73    20764.64      17.40               16.78      14.02   
std            39.45    17822.03       0.80                8.98       4.46   
min             0.00        0.00       3.00                1.00       0.00   
25%             1.00     6923.00      17.00                9.00      11.00   
50%             3.00    15689.00      17.00               18.00      14.00   
75%            20.00    29219.00      17.00               25.00      17.00   
max           200.00    68543.00      20.00               31.00      23.00   

l1                                                Session Duration             \
l2        minute      month     second       year             days      hours   
count 9775562.00 9775562.00 9775562.00 9775562.00       9775562.00 9775562.00   
mean       29.94       7.17      29.37    2019.17             0.00       0.20   
std        17.61       2.93      17.40       0.37             0.03       0.81   
min         0.00       1.00       0.00    2019.00             0.00       0.00   
25%        15.00       5.00      14.00    2019.00             0.00       0.00   
50%        30.00       7.00      29.00    2019.00             0.00       0.00   
75%        46.00      10.00      44.00    2019.00             0.00       0.00   
max        59.00      12.00      59.00    2020.00             6.00      23.00   

l1                              Utilization                               \
l2       minutes    seconds  Bytes Received      Bytes Sent Data Retries   
count 9775562.00 9775562.00      9775562.00      9775562.00   9775562.00   
mean       12.81       8.90     22606966.52      2537496.89     15298.11   
std        12.84      14.52    213604570.55    106651432.75     64036.03   
min         0.00       0.00            0.00            0.00         0.00   
25%         5.00       1.00            0.00            0.00       285.00   
50%         5.00       2.00            0.00            0.00      1714.00   
75%        15.00      10.00       686881.75       227125.00      9741.00   
max        59.00      59.00 212877464850.00 140114512584.00  25726438.00   

l1                                                                     
l2    Packets Received Packets Sent RSSI (dBm) RTS Retries   SNR (dB)  
count       9775562.00   9775562.00 9775562.00  9775562.00 9775562.00  
mean          17772.40      9629.41     -73.77        0.00      20.16  
std          168402.70    117053.67      15.54        0.00      12.43  
min               0.00         0.00    -128.00        0.00    -128.00  
25%               0.00         0.00     -80.00        0.00      11.00  
50%               0.00         0.00     -73.00        0.00      19.00  
75%            1179.00      1399.00     -65.00        0.00      28.00  
max       163194281.00 114895804.00       0.00        0.00     127.00

## 세션 개수

In [4]:
len(clients_df.index)

9775562

## AP별 세션 개수

In [5]:
clients_df.groupby([('AP','Name')]).count()

l1                  AP                          Association Time         \
l2          IP Address MAC Address Map Location              day   hour   
(AP, Name)                                                                
11S-103-2-1       3479        3479         3479             3479   3479   
11S-105-1-2       3462        3462         3462             3462   3462   
11S-105-2-4      17878       17878        17878            17878  17878   
11S-105-3-4       9945        9945         9945             9945   9945   
11S-105-4-4       9714        9714         9714             9714   9714   
...                ...         ...          ...              ...    ...   
SAP-94.236        6734        6734         6734             6734   6734   
SAP-94.237       18050       18050        18050            18050  18050   
SAP-94.238       10330       10330        10330            10330  10330   
SAP-94.239       12752       12752        12752            12752  12752   
SAP-94.240        9816        9816         9816             9816   9816   

l1                                              Client                 \
l2          minute  month second   year Association ID Authentication   
(AP, Name)                                                              
11S-103-2-1   3479   3479   3479   3479           3479           3479   
11S-105-1-2   3462   3462   3462   3462           3462           3462   
11S-105-2-4  17878  17878  17878  17878          17878          17878   
11S-105-3-4   9945   9945   9945   9945           9945           9945   
11S-105-4-4   9714   9714   9714   9714           9714           9714   
...            ...    ...    ...    ...            ...            ...   
SAP-94.236    6734   6734   6734   6734           6734           6734   
SAP-94.237   18050  18050  18050  18050          18050          18050   
SAP-94.238   10330  10330  10330  10330          10330          10330   
SAP-94.239   12752  12752  12752  12752          12752          12752   
SAP-94.240    9816   9816   9816   9816           9816           9816   

l1                                                                           \
l2          EAP Type Encryption Cipher Endpoint Type IP Address MAC Address   
(AP, Name)                                                                    
11S-103-2-1     3479              3479          3479       3479        3479   
11S-105-1-2     3462              3462          3462       3462        3462   
11S-105-2-4    17878             17878         17878      17878       17878   
11S-105-3-4     9945              9945          9945       9945        9945   
11S-105-4-4     9714              9714          9714       9714        9714   
...              ...               ...           ...        ...         ...   
SAP-94.236      6734              6734          6734       6734        6734   
SAP-94.237     18050             18050         18050      18050       18050   
SAP-94.238     10330             10330         10330      10330       10330   
SAP-94.239     12752             12752         12752      12752       12752   
SAP-94.240      9816              9816          9816       9816        9816   

l1                                                                 \
l2          Policy Type Protocol Radio Type   SSID   Type VLAN ID   
(AP, Name)                                                          
11S-103-2-1        3479     3479       3479   3479   3479    3479   
11S-105-1-2        3462     3462       3462   3462   3462    3462   
11S-105-2-4       17878    17878      17878  17878  17878   17878   
11S-105-3-4        9945     9945       9945   9945   9945    9945   
11S-105-4-4        9714     9714       9714   9714   9714    9714   
...                 ...      ...        ...    ...    ...     ...   
SAP-94.236         6734     6734       6734   6734   6734    6734   
SAP-94.237        18050    18050      18050  18050  18050   18050   
SAP-94.238        10330    10330      10330  10330  10330   